In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard

from keras.datasets import cifar10

In [ ]:
(X_train, y_train),(X_test, y_test) = cifar10.load_data()

In [ ]:
# floatに型変換
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 各画素値を正規化
X_train /= 255.0
X_test /= 255.0

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# モデルの定義
model = Sequential()

model.add(Conv2D(32,3,input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

adam = Adam(lr=1e-4)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=150, verbose=1, validation_split=0.1)

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
# モデルをプロット
plot_model(model, to_file='./model2.png')

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

# 画像を反転

In [ ]:
# モデルの定義
model = Sequential()

model.add(Conv2D(32,3,input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64,3))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

adam = Adam(lr=1e-4)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy"])

datagen = ImageDataGenerator(
                    rotation_range=90, # 90°まで回転
                    width_shift_range=0.1, # 水平方向にランダムでシフト
                    height_shift_range=0.1, # 垂直方向にランダムでシフト
                    channel_shift_range=50.0, # 色調をランダム変更
                    shear_range=0.39, # 斜め方向(pi/8まで)に引っ張る
                    horizontal_flip=True, # 垂直方向にランダムで反転
                    vertical_flip=True # 水平方向にランダムで反転
                    )
it_train = datagen.flow(X_train,y_train)

history = model.fit(X_train, y_train, batch_size=128, epochs=150, verbose=1, validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

# 1%増加

In [ ]:
!pip install dlt
import dlt

In [ ]:
folder = 'results'
if not os.path.exists(folder):
    os.makedirs(folder)

model.save(os.path.join(folder, 'my_model.h5'))

In [ ]:
from keras.models import load_model

cifar10_labels = np.array([
                'airplane',
                'automobile',
                'bird',
                'cat',
                'deer',
                'dog',
                'frog',
                'horse',
                'ship',
                'truck']
                )

def plot_image(X, label=None):
    print('☆テスト画像 (正解ラベル) : %s' % label)
    plt.imshow(X)
    plt.show()
    plt.clf()

if __name__=='__main__':
    # import data
    data = dlt.cifar.load_cifar10()
    X_train = data.train_images.reshape([-1, 32, 32, 3])
    X_test = data.test_images.reshape([-1, 32, 32, 3])
    X_train = data.train_images.astype('float32') / 255
    X_test = data.test_images.astype('float32') / 255

    # load model
    model = load_model(os.path.join('results', 'my_model.h5')) # 保存したモデルから読み込む

    # output
    n = 5
    preds = model.predict(X_test) # modelは学習させたもの
    cls= model.predict_classes(X_test)
    for i in range(n):
        true_label = cls[i]
        labels = cifar10_labels[true_label]
        plot_image(X_test[i], labels)
        print('{0} - 分類されたものは{1}です。精度は{2:.2f}%です。\n' .format(i, labels, preds[i][true_label]*100))
        print("################################################")